The python API is not currently stable, but you can render headlessly using `bpy` and `molecularnodes`.

This is an example of having an interactive jupyter notebook that uses Blender and Molecular Nodes in the background to render macromolecules from the PDB.

## Setup

Blender's python API is very verbose, and at times unintuitive, so we can create some helper functions for setting up the scene and rendering images.

#### Convenience Functions

In [1]:
#| label: setup
#| code-fold: true
import molecularnodes as mn
import bpy
import sys
import tempfile
import os
from IPython.display import display, Image
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import math
from mathutils import Vector


sys.stdout = open(os.devnull, 'w')
sys.stderr = open(os.devnull, 'w')
# mn.register()

def clear_scene():
    bpy.ops.object.select_all(action="DESELECT")
    bpy.ops.object.select_by_type(type="MESH")
    bpy.ops.object.delete()
    for node in bpy.data.node_groups:
        if node.type == "GEOMETRY":
            bpy.data.node_groups.remove(node)

def orient_camera(object, lens = 85, dof = True, f = 2, zoom = 0, focus = 0):
    object.select_set(True)
    camera = bpy.data.objects['Camera']
    distance = Vector(object.location) - Vector(camera.location)
    camera.data.lens = lens
    camera.data.dof.aperture_fstop = f
    camera.data.dof.focus_object = None
    bpy.ops.view3d.camera_to_view_selected()
    camera.data.lens = lens + zoom
    camera.data.dof.use_dof = dof
    camera.data.dof.focus_distance = distance.length + focus
    # camera.data.dof.focus_distance = 1.2

def render_image(engine = 'eevee', x = 1000, y = 500):
    # setup render engine
    if engine == "eevee":
        bpy.context.scene.render.engine = "BLENDER_EEVEE"
    elif engine == "cycles":
        
        bpy.context.scene.render.engine = "CYCLES"
        try:
            bpy.context.scene.cycles.device = "GPU"
        except:
            print("GPU Rendering not available")
    

    # Render
    with tempfile.TemporaryDirectory() as temp:

        path = os.path.join(temp, "test.png")
        bpy.context.scene.render.resolution_x = x
        bpy.context.scene.render.resolution_y = y
        bpy.context.scene.render.image_settings.file_format = "PNG"
        bpy.context.scene.render.filepath = path
        bpy.ops.render.render(write_still=True)
        display(Image(filename=path))

#### Scene Setup

In [2]:
#| label: setup-scene
#| code-fold: true

# load template scene with nice HDRI lighting
bpy.ops.wm.read_homefile(app_template = "MolecularNodes")

# change the background to a custom color
try:
    world_nodes = bpy.data.worlds['World Shader'].node_tree.nodes
    world_nodes['MN_world_shader'].inputs['BG Color'].default_value = mn.color.random_rgb()
except KeyError:
    print("Oh no, didn't set up the base scene.")

In [7]:
@interact
def choose_style(code: str = '8H1B', 
                 style = ['preset_1', 'preset_2', 'preset_3', 'preset_4', 'ribbon', 'cartoon', 'atoms', 'surface', 'ball_and_stick'], 
                 assembly = False, 
                 eevee = True,
                 width = (0, 3,  0.1), 
                 radius = (0, 3,  0.1), 
                 
                 quality = (int(0), int(5), int(1)),
                 seed = (0, 10, int(1)),
                 thickness = (0, 1,  0.1), 
                 rotate_x = (-math.pi, math.pi, 0.1),
                 rotate_y = (-math.pi, math.pi, 0.1),
                 rotate_z = (-math.pi, math.pi, 0.1),
                 attr = ['chain_id', 'res_name', 'entity_id', 'res_id', 'atomic_number', 'sec_struct'], 
                 lens = (16, 200, 5), 
                 zoom = (-100, 100, int(1)),
                 focus = (-2, 2, 0.02),
                 dof = True, 
                 f = (0.1, 10, 0.1)
                 ):
    mol = bpy.data.objects.get(code)
    if not mol or assembly:
        clear_scene()
        mol = mn.load.molecule_rcsb(code, center_molecule=(not assembly), starting_style=style, build_assembly=assembly)
        group = mol.modifiers['MolecularNodes'].node_group
    else:
        group = mol.modifiers['MolecularNodes'].node_group
        new_style = mn.nodes.styles_mapping[style]
        mn.nodes.append(new_style)
        for node in group.nodes:
            if node.name.startswith('MN_style') or node.name.startswith(".MN_style"):
                style_node = node
        style_node.node_tree = bpy.data.node_groups[new_style]
        style_node.inputs['Material'].default_value = bpy.data.materials['MN_atomic_material']
        group.links.new(
            group.nodes['MN_color_set'].outputs[0],
            style_node.inputs[0], 
        )
        group.links.new(
            style_node.outputs[0], 
            group.nodes['Group Output'].inputs[0]
        )
        
    # try:
    for node in group.nodes:
        for input in node.inputs:
            if input.name in ["Radius", "Width"]:
                input.default_value = width
            if input.name == "Thickness":
                input.default_value = thickness
            if "Radius" in input.name or "Radii" in input.name:
                input.default_value = radius
            if "Quality" in input.name or "Subd" in input.name or "subd" in input.name:
                input.default_value = quality
            if "Eevee" in input.name:
                input.default_value = eevee
            if input.name == "Seed":
                input.default_value = seed
    # group.nodes['MN_style_ribbon_protein'].inputs['Radius'].default_value = width
    # except:
    #     pass
    
    attribute = group.nodes['MN_color_attribute_random'].inputs.get('Attribute')
    if attribute:
        attribute.default_value = attr
    mol.select_set(True)
    
    mol.rotation_euler = (rotate_x, rotate_y, rotate_z)
    
    orient_camera(mol, lens = lens, dof = dof, f = f, zoom = zoom, focus = focus)
    render_image()

interactive(children=(Text(value='8H1B', description='code'), Dropdown(description='style', options=('preset_1…